In [41]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("johnsmith88/heart-disease-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\user\.cache\kagglehub\datasets\johnsmith88\heart-disease-dataset\versions\2


In [42]:
import os
import pandas as pd
import numpy as np
print(os.listdir(path))

['heart.csv']


In [43]:
file=os.path.join(path,'heart.csv')
df=pd.read_csv(file)
print(df)

      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      52    1   0       125   212    0        1      168      0      1.0   
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.4   

      slope  ca  thal  target  
0         2   2     3       0  
1         0

In [44]:
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
model1=DecisionTreeClassifier()
x=df.drop(columns=['target'])
y=df.iloc[:,-1]
X_tr,X_te,Y_tr,Y_te=train_test_split(x,y,test_size=0.2,random_state=10)
model1.fit(X_tr,Y_tr)
# Y_pred=model1.predict(X_te)

DecisionTreeClassifier()

VISUALISE TREE   UISNG GRAPHVIZ

In [45]:
import matplotlib.pyplot as plt
import graphviz
dot_data=export_graphviz(model1,out_file=None,
                        feature_names=x.columns,class_names=[str(c) for c in y.unique()],filled=True,rounded=True,special_characters=True)
graph=graphviz.Source(dot_data)

In [46]:
try:
    graph = graphviz.Source(dot_data)
    graph.render("decision_tree", format="png")  # Save as PNG
    graph.view()  # Open the file
except FileNotFoundError:
    print("❌ Graphviz 'dot' executable not found. Please install Graphviz and add it to PATH.")
except Exception as e:
    print(f"❌ An error occurred while rendering the decision tree: {e}")

Analyze overfitting and control tree depth
If train accuracy is ~100% but test accuracy drops a lot, you’re overfitting.

CONTROLLING DEPTH
clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X_train, y_train)
Here, we can limit tree levels upto 3 only.  Prevents it from memorizing too many fine details (noise).

In [47]:
print("Train Accuracy:", model1.score(X_tr,Y_tr))
print("Test Accuracy:", model1.score(X_te, Y_te))

Train Accuracy: 1.0
Test Accuracy: 1.0


TRAINING A RANDOM FOREST AND EVALUATING ACCURACY

In [48]:
from sklearn.ensemble import RandomForestClassifier
X_TRAIN,X_TEST,Y_TRAIN,Y_TEST=train_test_split(x,y,test_size=0.3,random_state=10)
model2=RandomForestClassifier()
model2.fit(X_TRAIN,Y_TRAIN)
y_p=model2.predict(X_TEST)

In [49]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(Y_TEST,y_p)
print(f'ACCURACY SCORE:{accuracy}')

ACCURACY SCORE:0.961038961038961


FEATURE IMPORTANCE

In [50]:
importances = model2.feature_importances_
feature_names =[f'feature{i}' for i in range(x.shape[1])]

# Show in a table
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(importance_df)


      Feature  Importance
2    feature2    0.130290
7    feature7    0.126687
9    feature9    0.114853
11  feature11    0.110827
12  feature12    0.108448
0    feature0    0.085764
3    feature3    0.073422
4    feature4    0.070513
8    feature8    0.059838
10  feature10    0.045421
1    feature1    0.043721
6    feature6    0.020780
5    feature5    0.009436


EVALUATION USING Cross-Validation

In [51]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=model2,X=X_TRAIN,y=Y_TRAIN,cv=10)
print(f"Accuracy:{accuracies.mean()*100:.2f}%")
print(f"Standard Deviation:{accuracies.std()*100:.2f}%")

Accuracy:98.61%
Standard Deviation:1.64%
